In [1]:
#Import packages
import pandas as pd
import numpy as np
import re
import warnings
import math
warnings.filterwarnings('ignore')

## GUIDE-IT

In [2]:
#IMPORT FILES NEEDED

demog = pd.read_csv("../Data Validation/GUIDE-IT/Raw Data/demog_ads.csv", sep=",", index_col='deidnum').sort_index()
corelab = pd.read_csv("../Data Validation/GUIDE-IT/Raw Data/adh_corelab_ads.csv", sep=",", index_col='deidnum').sort_index()
baseDF = pd.read_csv("../Data Validation/GUIDE-IT/Raw Data/base.csv", sep=",", index_col='deidnum').sort_index()
ef = pd.read_csv("../Data Validation/GUIDE-IT/Raw Data/ef.csv", sep=",", index_col='deidnum').sort_index()
exam = pd.read_csv("../Data Validation/GUIDE-IT/Raw Data/exam.csv", sep=",", index_col='deidnum').sort_index()
hosp = pd.read_csv("../Data Validation/GUIDE-IT/Raw Data/hosp1.csv", sep=",", index_col='deidnum').sort_index()
labs = pd.read_csv("../Data Validation/GUIDE-IT/Raw Data/labs_ads.csv", sep=",", index_col='deidnum').sort_index()
medhx = pd.read_csv("../Data Validation/GUIDE-IT/Raw Data/medhx.csv", sep=",", index_col='deidnum').sort_index()
meds = pd.read_csv("../Data Validation/GUIDE-IT/Raw Data/meds_ads.csv", sep=",", index_col='deidnum').sort_index()
allMeds = pd.read_csv("../Data Validation/GUIDE-IT/Raw Data/meds.csv", sep=",", index_col='deidnum').sort_index()
endpoints = pd.read_csv("../Data Validation/GUIDE-IT/Raw Data/best_endpoints_ads.csv", sep=",", index_col='deidnum').sort_index()


In [3]:
#DEMOGRAPHIC

cohort = pd.DataFrame()
cohort['ID'] = demog.index
cohort = cohort.set_index('ID')

#continue on from here

cohort['Age'] = demog['age'].tolist()

lst = []

for idx in demog.index:
    pt = demog.loc[idx]
    if pt['sex'] == 1: # male is 1, female is 0
        lst.append(1) #male is 1
    else:
        lst.append(2) #female 2
cohort['Gender'] = lst


lst = []

for idx in demog.index:
    pt = demog.loc[idx]
    if pt['whited'] == 1:#white is 1, all other races are 2
        lst.append(1)
    else:
        lst.append(2)
cohort['Race'] = lst

#weight    
cohort['Wt_B'] = demog['WEIGHTKG'].tolist()

lst = []
for idx in demog.index:
    pt = exam.loc[idx]
    try:
        pt = pt[pt['VISIT'] == 'evBLNFU']
        
        if pt['WEIGHTU'].item() == 'lbs':
            wt = pt['WEIGHT'].item() * 0.45359237
        else:
            wt = pt['WEIGHT'].item()
        
        lst.append(wt)
        
    except:
        lst.append(None)
        
cohort['Wt_D'] = lst 

#BMI
cohort['BMI_B'] = demog['bmi'].tolist() 

#EjF
base = []
disch = []
for idx in demog.index:
    pt = corelab.loc[idx]
    if type(pt) != pd.Series:
        ptb = pt[pt['visit'] == 'evBLN']
    else:
        ptb = pt
    
    base.append(ptb['efcontb'].item())
    
    try:
        ptd = pt[pt['visit'] == 'evBLNFU']
        disch.append(ptd['efcontb'].item())
    except:
        disch.append(None)

cohort['EjF_B'] = base
cohort['EjF_D'] = disch

#NYHA
lst = []
for idx in demog.index:
    pt = exam.loc[idx]
    
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLN']
            nh = pt['NYHACD'].item()
        except:
            nh = None
    else:
        nh = pt['NYHACD']
    
    lst.append(nh)
        
cohort['NYHA_B'] = lst 

lst = []
for idx in demog.index:
    pt = exam.loc[idx]
    
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLNFU']
            nh = pt['NYHACD'].item()
        except:
            nh = None
    else:
        nh = pt['NYHACD']
    
    lst.append(nh)
        
cohort['NYHA_D'] = lst 

#Hosp Days
cohort['TotalHospDays'] = endpoints['hosp730'].tolist() #how many days in hospitalization

cohort.head(5)

,Age,Gender,Race,Wt_B,Wt_D,BMI_B,EjF_B,EjF_D,NYHA_B,NYHA_D,TotalHospDays
ID,,,,,,,,,,,
11,57,2,2,88.905017,91.172066,33.643189,25.0,25.0,3.0,3.0,70
350,58,1,2,128.300000,NaN,39.449435,40.0,NaN,2.0,NaN,0
498,72,2,1,90.719405,NaN,24.998186,30.0,NaN,2.0,NaN,0
516,72,2,1,106.595000,NaN,33.718894,40.0,NaN,2.0,NaN,0
615,70,2,2,87.500000,NaN,34.179688,20.0,NaN,3.0,NaN,0


In [4]:
#conditions, Ischemic
lst = []
for idx in demog.index:
    pt = demog.loc[idx]
    medPt = medhx.loc[idx]
    if pt['ischemic'] == 1 or medPt['MI'] == 1 or medPt['HFETIOL'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
cohort['ISCH'] = lst

#Nonischemic
lst = []
for idx in medhx.index:
    pt = medhx.loc[idx]
    if pt['NISETIOL'] > 0 or medPt['HFETIOL'] == 2: #deal with mulitple non zero integers
        lst.append(1)
    else:
        lst.append(0)
cohort['NonISCH'] = lst

#AF
lst = []
for idx in demog.index:
    pt_demog = demog.loc[idx]
    pt_medhx = medhx.loc[idx]
    #pt_exam = exam.loc[idx3]
    if pt_demog['afibbase'] == 1 or pt_medhx['AFIB'] == 1: 
        lst.append(1)
    else:
        lst.append(0)

cohort['AF'] = lst

#AlchE
lst = []
for idx in medhx.index:
    pt = medhx.loc[idx]
    if pt['ALCOHOL'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['AlchE'] = lst

#COPD
lst = []

for idx in demog.index:
    pt_demog = demog.loc[idx]
    pt_medhx = demog.loc[idx]
    if pt_demog['COPD'] == 1 or pt_medhx['COPD'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['COPD'] = lst

#Depr 
lst = []

for idx in medhx.index:
    pt = medhx.loc[idx]
    if pt['DPRSNMED'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['DEPR'] = lst

#Diab
lst = []
for (idx1,idx2) in zip(demog.index, medhx.index):
    pt_demog = demog.loc[idx1]
    pt_medhx = medhx.loc[idx2]
    if pt_demog['DIABETES'] == 1 or pt_medhx['DIABETES'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['DIAB'] = lst

#HEPT
lst = []
for idx in medhx.index:
    pt_medhx = medhx.loc[idx]
    if pt_medhx['LIVER'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['HEPT'] = lst

#HTN
lst = []

for idx in medhx.index:
    pt = medhx.loc[idx]
    if pt['HIPRESSR'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['HTN'] = lst

#Malig
lst = []

for idx in medhx.index:
    pt = medhx.loc[idx]
    if pt['CANCER'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['MALIG'] = lst

#RENAL
lst = []
for idx2 in medhx.index:
    pt_medhx = medhx.loc[idx2]
    if pt_medhx['KIDNEY'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['RENAL'] = lst

#Smok
lst = []
for idx in medhx.index:
    pt = medhx.loc[idx]
    if pt['SMOKEHX'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['SMOKING'] = lst

#Stroke TIA
lst = []

for idx in medhx.index:
    pt = medhx.loc[idx]
    if pt['STROKE'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['StrokeTIA'] = lst

#VAHD
lst = []
for idx in medhx.index:
    pt = medhx.loc[idx]
    if pt['PAVD'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['VAHD'] = lst

#VF
lst = []

for idx in medhx.index:
    pt = medhx.loc[idx]
    if pt['VFIB'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['VF'] = lst

#VHD
# hosp1['MITRLVLV'] seems relevant but value error of series.
lst = []
for idx in medhx.index:
    pt = medhx.loc[idx]
    if pt['MITRVLV'] == 1 or pt['AORVLV'] == 1 or pt['OTHVLVSG'] ==1:
        lst.append(1)
    else:
        lst.append(0)
cohort['VHD'] = lst

#VT
lst = []
for idx in medhx.index:
    pt = medhx.loc[idx]
    if pt['LHC'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['VT'] = lst

#CABG
lst = []
for idx in medhx.index:
    pt = medhx.loc[idx]
    if pt['CABGS'] == 1 or pt['PCABG'] == 1 :
        lst.append(1)
    else:
        try:
            hpt = hosp.loc[idx]
            if hpt['CABG'].any() == 1:
                lst.append(0)
            else:
                lst.append(0)
        except:
            lst.append(0)
            
cohort['CABG'] = lst

#HTRANS
lst = []
for idx in demog.index:
    if idx in hosp.index:
        pt = hosp.loc[idx]
        if pt['PROHTRAN'].any() == 1:
            lst.append(1)
        else:
            lst.append(0)
    else:
        lst.append(0)
cohort['HTRANS'] = lst

#ICD and pace
icd = []
pace = []
for idx in demog.index:
    p = 0
    i = 0
    medpt = medhx.loc[idx]
    if medpt['ICD'] == 2 or medpt['ICD'] == 3:
        p = 1
    if medpt['ICD'] == 1 or medpt['ICD'] == 4:
        i = 1
    
    if idx in hosp.index:
        pt = hosp.loc[idx]
        if pt['PACTYPE'].any() == 1 or pt['PACTYPE'].any() == 3:
            p = 1

        if pt['PACTYPE'].any() == 2 or pt['PACTYPE'].any() == 4:
            i = 1

    pace.append(p)
    icd.append(i)
        
cohort['ICD'] = icd
cohort['PACE'] = pace

#PTCI
lst = []
for idx in demog.index:
    if idx in hosp.index:
        pt = hosp.loc[idx]
        if pt['PCI'].any() == 1:
            lst.append(1)
        else:
            lst.append(0)
    else:
        lst.append(0)
cohort['PTCI'] = lst

cohort.head(5)

,Age,Gender,Race,Wt_B,Wt_D,BMI_B,EjF_B,EjF_D,NYHA_B,NYHA_D,...,StrokeTIA,VAHD,VF,VHD,VT,CABG,HTRANS,ICD,PACE,PTCI
ID,,,,,,,,,,,,,,,,,,,,,
11,57,2,2,88.905017,91.172066,33.643189,25.0,25.0,3.0,3.0,...,0,0,0,0,1,0,0,0,1,0
350,58,1,2,128.300000,NaN,39.449435,40.0,NaN,2.0,NaN,...,0,0,0,0,0,0,0,1,0,0
498,72,2,1,90.719405,NaN,24.998186,30.0,NaN,2.0,NaN,...,0,0,1,0,0,0,0,1,0,0
516,72,2,1,106.595000,NaN,33.718894,40.0,NaN,2.0,NaN,...,0,0,0,0,1,1,0,0,1,1
615,70,2,2,87.500000,NaN,34.179688,20.0,NaN,3.0,NaN,...,0,0,0,0,1,1,0,0,0,0


In [5]:
#exercise
cohort['SixFtWlk'] = baseDF['WALKDIST'].tolist()

In [6]:
# Labs
#BUN
lst = []
for idx in demog.index:
    pt = labs.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLN']
            nh = pt['BUNRSLT'].item()
        except:
            nh = None
    else:
        nh = pt['BUNRSLT']
    lst.append(nh)
        
cohort['BUN_B'] = lst 

lst = []
for idx in demog.index:
    pt = labs.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLNFU']
            nh = pt['BUNRSLT'].item()
        except:
            nh = None
    else:
        nh = pt['BUNRSLT']
    lst.append(nh)
        
cohort['BUN_D'] = lst

#CRT
lst = []
for idx in demog.index:
    pt = labs.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLN']
            nh = pt['CRTRSLT'].item()
        except:
            nh = None
    else:
        nh = pt['CRTRSLT']
    lst.append(nh)
        
cohort['CRT_B'] = lst 

lst = []
for idx in demog.index:
    pt = labs.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLNFU']
            nh = pt['CRTRSLT'].item()
        except:
            nh = None
    else:
        nh = pt['CRTRSLT']
    lst.append(nh)
        
cohort['CRT_D'] = lst

#HEC
lst = []
for idx in demog.index:
    pt = labs.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLN']
            nh = pt['HEMARSLT'].item()
        except:
            nh = None
    else:
        nh = pt['HEMARSLT']
    lst.append(nh)
        
cohort['HEC_B'] = lst 

lst = []
for idx in demog.index:
    pt = labs.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLNFU']
            nh = pt['HEMARSLT'].item()
        except:
            nh = None
    else:
        nh = pt['HEMARSLT']
    lst.append(nh)
        
cohort['HEC_D'] = lst

#HEM
lst = []
for idx in demog.index:
    pt = labs.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLN']
            nh = pt['HEMORSLT'].item()
        except:
            nh = None
    else:
        nh = pt['HEMORSLT']
    lst.append(nh)
        
cohort['HEM_B'] = lst 

lst = []
for idx in demog.index:
    pt = labs.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLNFU']
            nh = pt['HEMORSLT'].item()
        except:
            nh = None
    else:
        nh = pt['HEMORSLT']
    lst.append(nh)
        
cohort['HEM_D'] = lst

#PLA
lst = []
for idx in demog.index:
    pt = labs.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLN']
            nh = pt['PLATRSLT'].item()
        except:
            nh = None
    else:
        nh = pt['PLATRSLT']
    lst.append(nh)
        
cohort['PLA_B'] = lst 

lst = []
for idx in demog.index:
    pt = labs.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLNFU']
            nh = pt['PLATRSLT'].item()
        except:
            nh = None
    else:
        nh = pt['PLATRSLT']
    lst.append(nh)
        
cohort['PLA_D'] = lst

#POT
lst = []
for idx in demog.index:
    pt = labs.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLN']
            nh = pt['POTRSLT'].item()
        except:
            nh = None
    else:
        nh = pt['POTRSLT']
    lst.append(nh)
        
cohort['POT_B'] = lst 

lst = []
for idx in demog.index:
    pt = labs.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLNFU']
            nh = pt['POTRSLT'].item()
        except:
            nh = None
    else:
        nh = pt['POTRSLT']
    lst.append(nh)
        
cohort['POT_D'] = lst

#SOD
lst = []
for idx in demog.index:
    pt = labs.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLN']
            nh = pt['sodrslt'].item()
        except:
            nh = None
    else:
        nh = pt['sodrslt']
    lst.append(nh)
        
cohort['SOD_B'] = lst 

lst = []
for idx in demog.index:
    pt = labs.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLNFU']
            nh = pt['sodrslt'].item()
        except:
            nh = None
    else:
        nh = pt['sodrslt']
    lst.append(nh)
        
cohort['SOD_D'] = lst

#WBC
lst = []
for idx in demog.index:
    pt = labs.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLN']
            nh = pt['WBCRSLT'].item()
        except:
            nh = None
    else:
        nh = pt['WBCRSLT']
    lst.append(nh)
        
cohort['WBC_B'] = lst 

lst = []
for idx in demog.index:
    pt = labs.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLNFU']
            nh = pt['WBCRSLT'].item()
        except:
            nh = None
    else:
        nh = pt['WBCRSLT']
    lst.append(nh)
        
cohort['WBC_D'] = lst

cohort

,Age,Gender,Race,Wt_B,Wt_D,BMI_B,EjF_B,EjF_D,NYHA_B,NYHA_D,...,HEM_B,HEM_D,PLA_B,PLA_D,POT_B,POT_D,SOD_B,SOD_D,WBC_B,WBC_D
ID,,,,,,,,,,,,,,,,,,,,,
11,57,2,2,88.905017,91.172066,33.643189,25.0,25.0,3.0,3.0,...,NaN,NaN,NaN,NaN,4.6,4.5,134.0,136.0,NaN,NaN
350,58,1,2,128.300000,NaN,39.449435,40.0,NaN,2.0,NaN,...,13.0,NaN,294.0,NaN,3.8,NaN,137.0,NaN,8.4,NaN
498,72,2,1,90.719405,NaN,24.998186,30.0,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,4.9,NaN,139.0,NaN,NaN,NaN
516,72,2,1,106.595000,NaN,33.718894,40.0,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,4.3,NaN,144.0,NaN,NaN,NaN
615,70,2,2,87.500000,NaN,34.179688,20.0,NaN,3.0,NaN,...,13.0,NaN,275.0,NaN,4.2,NaN,130.0,NaN,7.3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99355,46,2,2,75.750703,NaN,23.379847,15.0,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,3.4,NaN,128.0,NaN,NaN,NaN
99624,52,2,2,70.761136,NaN,19.498585,25.0,NaN,2.0,NaN,...,13.2,NaN,257.0,NaN,5.1,NaN,133.0,NaN,5.1,NaN
99676,74,1,2,96.434727,NaN,36.474754,20.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,4.5,NaN,141.0,NaN,NaN,NaN


In [7]:
#MEDS

#ACE
lst = []
for idx in demog.index:
    pt = meds.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLN']
            if pt['ACE'].item() == 1 or pt['ARB'].item() == 1:
                nh = 1
            else:
                nh = 0
        except:
            nh = None
    else:
        if pt['ACE'] == 1 or pt['ARB'] == 1:
            nh = 1
        else:
            nh = 0
    lst.append(nh)
        
cohort['ACE_B'] = lst 

#BET
lst = []
for idx in demog.index:
    pt = meds.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLN']
            nh = pt['BETAB'].item()
        except:
            nh = None
    else:
        nh = pt['BETAB']
    lst.append(nh)
        
cohort['BET_B'] = lst 

#NIT
lst = []
for idx in demog.index:
    pt = allMeds.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLN']
            nh = pt['NITR'].item()
        except:
            nh = None
    else:
        nh = pt['NITR']
    lst.append(nh)
        
cohort['NIT_B'] = lst 

#NIT
lst = []
for idx in demog.index:
    pt = meds.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLN']
            if pt['diurdose'].item() > 0:
                nh = 1
            else:
                nh = 0
        except:
            nh = None
    else:
        if pt['diurdose'] > 0:
            nh = 1
        else:
            nh = 0
    lst.append(nh)
        
cohort['DIUR_B'] = lst 


cohort

,Age,Gender,Race,Wt_B,Wt_D,BMI_B,EjF_B,EjF_D,NYHA_B,NYHA_D,...,POT_B,POT_D,SOD_B,SOD_D,WBC_B,WBC_D,ACE_B,BET_B,NIT_B,DIUR_B
ID,,,,,,,,,,,,,,,,,,,,,
11,57,2,2,88.905017,91.172066,33.643189,25.0,25.0,3.0,3.0,...,4.6,4.5,134.0,136.0,NaN,NaN,1.0,1.0,0.0,1.0
350,58,1,2,128.300000,NaN,39.449435,40.0,NaN,2.0,NaN,...,3.8,NaN,137.0,NaN,8.4,NaN,1.0,1.0,1.0,1.0
498,72,2,1,90.719405,NaN,24.998186,30.0,NaN,2.0,NaN,...,4.9,NaN,139.0,NaN,NaN,NaN,1.0,1.0,0.0,1.0
516,72,2,1,106.595000,NaN,33.718894,40.0,NaN,2.0,NaN,...,4.3,NaN,144.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0
615,70,2,2,87.500000,NaN,34.179688,20.0,NaN,3.0,NaN,...,4.2,NaN,130.0,NaN,7.3,NaN,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99355,46,2,2,75.750703,NaN,23.379847,15.0,NaN,4.0,NaN,...,3.4,NaN,128.0,NaN,NaN,NaN,1.0,0.0,0.0,1.0
99624,52,2,2,70.761136,NaN,19.498585,25.0,NaN,2.0,NaN,...,5.1,NaN,133.0,NaN,5.1,NaN,0.0,1.0,0.0,1.0
99676,74,1,2,96.434727,NaN,36.474754,20.0,NaN,1.0,NaN,...,4.5,NaN,141.0,NaN,NaN,NaN,1.0,1.0,0.0,1.0


In [8]:
#HR
lst = []
for idx in demog.index:
    pt = exam.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLN']
            nh = pt['HRVALUE'].item()
        except:
            nh = None
    else:
        nh = pt['HRVALUE']
    lst.append(nh)
        
cohort['HRTRT_B'] = lst 

lst = []
for idx in demog.index:
    pt = exam.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLNFU']
            nh = pt['HRVALUE'].item()
        except:
            nh = None
    else:
        nh = pt['HRVALUE']
    lst.append(nh)
        
cohort['HRTRT_D'] = lst 

#BPSYS
lst = []
for idx in demog.index:
    pt = exam.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLN']
            nh = pt['BPSYS'].item()
        except:
            nh = None
    else:
        nh = pt['BPSYS']
    lst.append(nh)
        
cohort['BPSYS_B'] = lst 

lst = []
for idx in demog.index:
    pt = exam.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLNFU']
            nh = pt['BPSYS'].item()
        except:
            nh = None
    else:
        nh = pt['BPSYS']
    lst.append(nh)
        
cohort['BPSYS_D'] = lst 

#BPDIAS
lst = []
for idx in demog.index:
    pt = exam.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLN']
            nh = pt['BPDIA'].item()
        except:
            nh = None
    else:
        nh = pt['BPDIA']
    lst.append(nh)
        
cohort['BPDIAS_B'] = lst 

lst = []
for idx in demog.index:
    pt = exam.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['VISIT'] == 'evBLNFU']
            nh = pt['BPDIA'].item()
        except:
            nh = None
    else:
        nh = pt['BPDIA']
    lst.append(nh)
        
cohort['BPDIAS_D'] = lst 


cohort

,Age,Gender,Race,Wt_B,Wt_D,BMI_B,EjF_B,EjF_D,NYHA_B,NYHA_D,...,ACE_B,BET_B,NIT_B,DIUR_B,HRTRT_B,HRTRT_D,BPSYS_B,BPSYS_D,BPDIAS_B,BPDIAS_D
ID,,,,,,,,,,,,,,,,,,,,,
11,57,2,2,88.905017,91.172066,33.643189,25.0,25.0,3.0,3.0,...,1.0,1.0,0.0,1.0,88.0,88.0,102.0,101.0,69.0,69.0
350,58,1,2,128.300000,NaN,39.449435,40.0,NaN,2.0,NaN,...,1.0,1.0,1.0,1.0,82.0,NaN,151.0,NaN,106.0,NaN
498,72,2,1,90.719405,NaN,24.998186,30.0,NaN,2.0,NaN,...,1.0,1.0,0.0,1.0,72.0,NaN,115.0,NaN,67.0,NaN
516,72,2,1,106.595000,NaN,33.718894,40.0,NaN,2.0,NaN,...,1.0,1.0,1.0,1.0,80.0,NaN,100.0,NaN,60.0,NaN
615,70,2,2,87.500000,NaN,34.179688,20.0,NaN,3.0,NaN,...,1.0,1.0,0.0,1.0,90.0,NaN,132.0,NaN,78.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99355,46,2,2,75.750703,NaN,23.379847,15.0,NaN,4.0,NaN,...,1.0,0.0,0.0,1.0,90.0,NaN,101.0,NaN,71.0,NaN
99624,52,2,2,70.761136,NaN,19.498585,25.0,NaN,2.0,NaN,...,0.0,1.0,0.0,1.0,94.0,NaN,132.0,NaN,84.0,NaN
99676,74,1,2,96.434727,NaN,36.474754,20.0,NaN,1.0,NaN,...,1.0,1.0,0.0,1.0,87.0,NaN,154.0,NaN,62.0,NaN


In [9]:

#Calculate all composite hemo for you --> change data file name to cohort for all these
idx = sorted(set(cohort.index))

#MAP
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = t['BPSYS_B'] + ((2 * t['BPDIAS_B']) / 3)
    f = t['BPSYS_D'] + ((2 * t['BPDIAS_D']) / 3)
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
cohort['MAP_B'] = base
cohort['MAP_D'] = final

    
#PP
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = t['BPSYS_B'] - t['BPDIAS_B']
    f = t['BPSYS_D'] - t['BPDIAS_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
cohort['PP_B'] = base
cohort['PP_D'] = final

#PPP
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = t['PP_B'] / t['BPSYS_B']
    f = t['PP_D'] / t['BPSYS_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
cohort['PPP_B'] = base
cohort['PPP_D'] = final



#PPRatio
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = t['PP_B'] / t['HRTRT_B']
    f = t['PP_D'] / t['HRTRT_D']
    c = f - b

    base.append(b)
    final.append(f)
    
cohort['PPRatio_B'] = base
cohort['PPRatio_D'] = final




cohort.head(5)

,Age,Gender,Race,Wt_B,Wt_D,BMI_B,EjF_B,EjF_D,NYHA_B,NYHA_D,...,BPDIAS_B,BPDIAS_D,MAP_B,MAP_D,PP_B,PP_D,PPP_B,PPP_D,PPRatio_B,PPRatio_D
ID,,,,,,,,,,,,,,,,,,,,,
11,57,2,2,88.905017,91.172066,33.643189,25.0,25.0,3.0,3.0,...,69.0,69.0,148.000000,147.0,33.0,32.0,0.323529,0.316832,0.375000,0.363636
350,58,1,2,128.300000,NaN,39.449435,40.0,NaN,2.0,NaN,...,106.0,NaN,221.666667,NaN,45.0,NaN,0.298013,NaN,0.548780,NaN
498,72,2,1,90.719405,NaN,24.998186,30.0,NaN,2.0,NaN,...,67.0,NaN,159.666667,NaN,48.0,NaN,0.417391,NaN,0.666667,NaN
516,72,2,1,106.595000,NaN,33.718894,40.0,NaN,2.0,NaN,...,60.0,NaN,140.000000,NaN,40.0,NaN,0.400000,NaN,0.500000,NaN
615,70,2,2,87.500000,NaN,34.179688,20.0,NaN,3.0,NaN,...,78.0,NaN,184.000000,NaN,54.0,NaN,0.409091,NaN,0.600000,NaN


In [10]:
#Make separate datasets
#No hemo data so no hemo dataset

#All Data - conditions
varz = ['Age', 'Gender', 'Race', 'Wt', 'BMI', 
       'InitialHospDays', 'TotalHospDays', 'NYHA', 'MLHFS', 'AF',
       'AlchE', 'ANGP', 'ARRH', 'CARREST', 'CVD', 'COPD', 'DEPR', 'DIAB',
       'GOUT', 'HEPT', 'HTN', 'MALIG', 'RENAL', 'SMOKING', 'STERD',
       'StrokeTIA', 'VAHD', 'VF', 'VHD', 'VT', 'ISCH', 'NonISCH', 'CABG',
       'HTRANS', 'ICD', 'PACE', 'PTCI', 'SixFtWlk', 'VO2','ALB', 'ALT', 'AST', 'BUN',
       'CRT', 'DIAL', 'HEC', 
       'HEM','PLA', 'POT',  'SOD',
       'TALB', 'TOTP', 'WBC', 'ACE','BET', 'NIT', 'DIUR',
       'EjF', 'BPDIAS', 'BPSYS', 'HR',
       'PV', 'MAP', 'PP', 'PPP', 'PPRatio']



giantList = []
for i in range(len(cohort)):
    df = cohort.iloc[i]
    row0 = []
    row1 = []
    row0.append(df.name)
    row1.append(df.name)
    for col in varz:
        if col in cohort.columns: #value for both
            row0.append(df[col])
            row1.append(df[col])
        elif col + "_B" in cohort.columns and col + "_D" in cohort.columns:
            row0.append(df[col+"_B"])
            row1.append(df[col+"_D"])
        elif col + "_B" in cohort.columns and not col + "_D" in cohort.columns:
            row0.append(df[col+"_B"])
            row1.append(0)
        elif not col + "_B" in cohort.columns and col + "_D" in cohort.columns:
            row0.append(0)
            row1.append(df[col+"_D"])
        else:
            row0.append(0)
            row1.append(0)
    
    giantList.append(row0)
    giantList.append(row1)

varz.insert(0, 'ID')
allData = pd.DataFrame(giantList, columns=varz).set_index('ID', drop=True)
    
allData

,Age,Gender,Race,Wt,BMI,InitialHospDays,TotalHospDays,NYHA,MLHFS,AF,...,DIUR,EjF,BPDIAS,BPSYS,HR,PV,MAP,PP,PPP,PPRatio
ID,,,,,,,,,,,,,,,,,,,,,
11,57.0,2.0,2.0,88.905017,33.643189,0,70.0,3.0,0,0.0,...,1.0,25.0,69.0,102.0,0,0,148.000000,33.0,0.323529,0.375000
11,57.0,2.0,2.0,91.172066,0.000000,0,70.0,3.0,0,0.0,...,0.0,25.0,69.0,101.0,0,0,147.000000,32.0,0.316832,0.363636
350,58.0,1.0,2.0,128.300000,39.449435,0,0.0,2.0,0,1.0,...,1.0,40.0,106.0,151.0,0,0,221.666667,45.0,0.298013,0.548780
350,58.0,1.0,2.0,NaN,0.000000,0,0.0,NaN,0,1.0,...,0.0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
498,72.0,2.0,1.0,90.719405,24.998186,0,0.0,2.0,0,0.0,...,1.0,30.0,67.0,115.0,0,0,159.666667,48.0,0.417391,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99676,74.0,1.0,2.0,NaN,0.000000,0,0.0,NaN,0,0.0,...,0.0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
99887,60.0,2.0,2.0,76.204300,24.105394,0,0.0,1.0,0,0.0,...,1.0,10.0,88.0,132.0,0,0,190.666667,44.0,0.333333,0.463158
99887,60.0,2.0,2.0,NaN,0.000000,0,0.0,NaN,0,0.0,...,0.0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN


In [11]:
#save to file
allData.to_csv("../Data Validation/GUIDE-IT/Original DataFrames/AllDataGUIDE-IT.csv")

In [12]:
#Make Labels
labels = pd.DataFrame()
labels['ID'] = demog.index
labels = labels.set_index('ID')

dthList =  []
rhspList = []
idx = sorted(set(demog.index))
for i in idx:
    t = endpoints.loc[i]
    c = cohort.loc[i]
    #Death outcome
    dth = t["DEATHADJ"] + c['HTRANS'] #Death is death, LVAD or Transplant 
    if dth >= 1:
        dthList.append(1)
    else:
        dthList.append(0)
        
    #Rehosp outcome
    rhsp = t['hospcvnum'] #Any rehospitalization
    if rhsp >= 1:
        rhspList.append(1)
    else:
        rhspList.append(0)
        
labels['Death'] = dthList
labels['Rehosp'] = rhspList
#no readmission data

labels

,Death,Rehosp
ID,,
11,0,1
350,0,0
498,0,0
516,0,0
615,0,0
...,...,...
99355,1,1
99624,1,1
99676,0,0


In [13]:
labels.to_csv("../Data Validation/GUIDE-IT/Original DataFrames/LabelsGUIDE-IT.csv")